In [1]:
import pandas as pd
import requests
import io
import re
import numpy as np
import ccxt
pd.options.display.max_rows = 50

### Section I. Ingest data

In [2]:
df = pd.read_csv("Candles.csv")

#### separate out the months, days, years, and hours for more finer grained analysis

In [3]:
df = df.join(df['__time'].str.split(' ', expand=True).add_prefix('time'))

#### split the raw_pairs to find potential triangular trade opportunities

In [4]:
df = df.join(df['raw_pair'].str.split('/', expand=True).add_prefix('raw_pair'))

#### rename columns for easier readibility 

In [5]:
df.rename(columns ={'time0': 'weekday', 'time1': 'month', 'time2': 'day', 'time3':'year', 'time4':'time', 'time5':'24_hour_time'}, inplace =True)

#### the df contains pair trades and then other types of trades
#### separate out the pair trades from the non pair trades as the strategies will differ

In [6]:
mask = df.raw_pair.str.contains('/', na=False)

In [7]:
df_pairs = df[mask]
df_no_pairs = df[~mask]

In [25]:
df_pairs

,__time,marketplace,raw_pair,price_open,price_high,price_low,price_close,price_vwap,trade_count,volume,...,month,day,year,time,24_hour_time,time6,time7,time8,raw_pair0,raw_pair1
0,Fri Mar 15 2019 18:00:00 GMT-0600 (Mountain Da...,bitfinex,ZCN/ETH,0.000945,0.000945,0.000945,0.000945,0.000945,1,4.000000e+02,...,Mar,15,2019,18:00:00,GMT-0600,(Mountain,Daylight,Time),ZCN,ETH
1,Fri Mar 15 2019 18:00:00 GMT-0600 (Mountain Da...,binance,ZRX/BTC,0.000069,0.000069,0.000067,0.000067,0.000068,11570,5.795909e+06,...,Mar,15,2019,18:00:00,GMT-0600,(Mountain,Daylight,Time),ZRX,BTC
2,Fri Mar 15 2019 18:00:00 GMT-0600 (Mountain Da...,binance,ZRX/ETH,0.001977,0.001977,0.001896,0.001906,0.001922,1963,4.492480e+05,...,Mar,15,2019,18:00:00,GMT-0600,(Mountain,Daylight,Time),ZRX,ETH
3,Fri Mar 15 2019 18:00:00 GMT-0600 (Mountain Da...,binance,ZRX/BNB,0.018077,0.018077,0.016831,0.016831,0.017490,275,1.733287e+05,...,Mar,15,2019,18:00:00,GMT-0600,(Mountain,Daylight,Time),ZRX,BNB
4,Fri Mar 15 2019 18:00:00 GMT-0600 (Mountain Da...,binance,ZRX/USDT,0.271170,0.274445,0.267342,0.269371,0.270710,3402,2.161824e+06,...,Mar,15,2019,18:00:00,GMT-0600,(Mountain,Daylight,Time),ZRX,USDT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1167804,Sat Mar 13 2021 17:00:00 GMT-0700 (Mountain St...,coinbase,DNT/USDC,0.303575,0.331158,0.296146,0.305092,0.312477,55133,8.071446e+07,...,Mar,13,2021,17:00:00,GMT-0700,(Mountain,Standard,Time),DNT,USDC
1167805,Sat Mar 13 2021 17:00:00 GMT-0700 (Mountain St...,binance,RLC/BTC,0.000039,0.000040,0.000038,0.000038,0.000039,7002,8.743760e+05,...,Mar,13,2021,17:00:00,GMT-0700,(Mountain,Standard,Time),RLC,BTC
1167806,Sat Mar 13 2021 17:00:00 GMT-0700 (Mountain St...,binance,RLC/ETH,0.001239,0.001272,0.001221,0.001228,0.001246,1210,1.006669e+05,...,Mar,13,2021,17:00:00,GMT-0700,(Mountain,Standard,Time),RLC,ETH
1167807,Sat Mar 13 2021 17:00:00 GMT-0700 (Mountain St...,binance,RLC/USDT,2.380457,2.402504,2.281474,2.302935,2.341332,40250,2.665022e+06,...,Mar,13,2021,17:00:00,GMT-0700,(Mountain,Standard,Time),RLC,USDT


#### exploratory analysis

In [9]:
df_no_pairs.to_csv('no_pairs.csv')
df_pairs.to_csv('pairs.csv')

In [11]:
time_pairs_df.max().to_csv('grouped_by_ticker_max.csv')

In [34]:
time_pairs_df.min().to_csv('grouped_by_ticker_min.csv')

In [12]:
df_pairs.groupby(['__time','raw_pair0', 'raw_pair1']).count().to_csv('grouped_by_time_ticker_count.csv')

### arbitraging by marketplace: opportunities are identified by time and raw_pair so group by those

for each raw_pair and time instance, select the max high price and minimum low price to identify opportunities for gain

In [13]:
df_pairs_grouped_time = df_pairs.groupby(['__time','raw_pair']).agg({'price_high':['count', 'max'], 'price_low':['min']})

In [162]:
df_pairs_grouped_time.to_csv('grouped_pairs_time_for_filter_count.csv')

In [24]:
df_pairs_grouped_time

price_high  \
                                                                  count   
__time                                             raw_pair               
Fri Apr 03 2020 18:00:00 GMT-0600 (Mountain Day... 1ST/BTC            1   
                                                   ABYSS/BTC          1   
                                                   ABYSS/USD          1   
                                                   ADA/BNB            1   
                                                   ADA/BTC            3   
...                                                                 ...   
Wed Sep 30 2020 18:00:00 GMT-0600 (Mountain Day... ZRX/BUSD           1   
                                                   ZRX/ETH            4   
                                                   ZRX/EUR            1   
                                                   ZRX/USD            4   
                                                   ZRX/USDT           4   

                                                                            \
                                                                       max   
__time                                             raw_pair                  
Fri Apr 03 2020 18:00:00 GMT-0600 (Mountain Day... 1ST/BTC    6.140000e-06   
                                                   ABYSS/BTC  9.281806e-07   
                                                   ABYSS/USD  6.296640e-03   
                                                   ADA/BNB    2.420334e-03   
                                                   ADA/BTC    4.809787e-06   
...                                                                    ...   
Wed Sep 30 2020 18:00:00 GMT-0600 (Mountain Day... ZRX/BUSD   4.253844e-01   
                                                   ZRX/ETH    1.166365e-03   
                                                   ZRX/EUR    3.634077e-01   
                                                   ZRX/USD    4.252473e-01   
                                                   ZRX/USDT   4.252620e-01   

                                                                 price_low  
                                                                       min  
__time                                             raw_pair                 
Fri Apr 03 2020 18:00:00 GMT-0600 (Mountain Day... 1ST/BTC    6.140000e-06  
                                                   ABYSS/BTC  8.989516e-07  
                                                   ABYSS/USD  6.106800e-03  
                                                   ADA/BNB    2.308492e-03  
                                                   ADA/BTC    4.703248e-06  
...                                                                    ...  
Wed Sep 30 2020 18:00:00 GMT-0600 (Mountain Day... ZRX/BUSD   3.935192e-01  
                                                   ZRX/ETH    1.114975e-03  
                                                   ZRX/EUR    3.370795e-01  
                                                   ZRX/USD    3.926828e-01  
                                                   ZRX/USDT   3.920103e-01  

[718905 rows x 3 columns]

filter out opportunities by limiting it to only rows where the raw pair and time instance appears more than once otherwise there's no opportunity

In [61]:
mask_count = df_pairs_grouped_time[("price_high", "count")]>1

narrow down the df by keeping only time and raw pair rows that happen more than once

In [15]:
greater_than_one_count = df_pairs_grouped_time[mask_count]

In [16]:
greater_than_one_reset = greater_than_one_count.reset_index()

In [17]:
new_df = pd.DataFrame(greater_than_one_reset.to_records())

In [18]:
new_df.columns = ['index', 'time', 'raw_pair', 'count', 'max_high_price', 'min_low_price']

In [19]:
new_df['price_differential'] = new_df['max_high_price'] - new_df['min_low_price']

In [29]:
new_df.columns

Index(['index', 'time', 'raw_pair', 'count', 'max_high_price', 'min_low_price',
       'price_differential', 'arb_opp'],
      dtype='object')

In [ ]:
create arb opp to denote the arbitrage opportunity when it exists to join it back to original dataframe

In [33]:
new_df['time'] = new_df['time'].astype(str)
new_df['max_high_price'] = new_df['max_high_price'].astype(str)
new_df['min_low_price'] = new_df['min_low_price'].astype(str)
new_df['arb_opp_sell'] = new_df['raw_pair'] + " " + new_df['time'].astype(str) + " " +new_df['max_high_price']
new_df['arb_opp_buy'] = new_df['raw_pair'] + " " + new_df['time'].astype(str) + " " +new_df['min_low_price']

In [39]:
df['price_high'] = df['price_high'].astype(str)
df['arb_opp_sell'] = df['raw_pair'] +" "+ df['__time']+ " " +df['price_high']

In [40]:
df['price_low'] = df['price_low'].astype(str)
df['arb_opp_buy'] = df['raw_pair'] +" "+ df['__time']+ " " +df['price_low']

In [79]:
result_buy = pd.merge(new_df, df, how='inner', left_on = 'arb_opp_buy', right_on = 'arb_opp_buy')

In [80]:
result_sell = pd.merge(new_df, df, how='inner', left_on = 'arb_opp_sell', right_on = 'arb_opp_sell')

In [81]:
frames = [result_sell, result_buy]

In [82]:
result_all = pd.concat(frames)

look at how many opportunities there are 

In [54]:
result_all.shape

(316139, 39)

In [83]:
deduped_result = result_all.sort_values(by=['price_differential', 'volume_max'], ascending = False).drop_duplicates(subset = ['__time', 'marketplace', 'price_high'])

In [84]:
deduped_result.shape

(254400, 39)

In [85]:
deduped_result_new_eliminate_zero = deduped_result_new.loc[deduped_result_new['price_differential']!=0]

In [86]:
deduped_result_new_eliminate_zero = deduped_result_new_eliminate_zero.drop(columns = ['index'])

In [87]:
deduped_result_new_eliminate_zero.to_csv('arb_opp_exam.csv')

In [94]:
deduped_result_new_eliminate_zero.columns

Index(['time_x', 'raw_pair_x', 'count', 'max_high_price', 'min_low_price',
       'price_differential', 'arb_opp_x', 'arb_opp_sell', 'arb_opp_buy_x',
       '__time', 'marketplace', 'raw_pair_y', 'price_open', 'price_high',
       'price_low', 'price_close', 'price_vwap', 'trade_count', 'volume',
       'volume_avg', 'volume_max', 'volume_min', 'weekday', 'month', 'day',
       'year', 'time_y', '24_hour_time', 'time6', 'time7', 'time8',
       'raw_pair0', 'raw_pair1', 'arb_opp_y', 'arb_opp_buy_y',
       'arb_opp_sell_x', 'arb_opp_buy', 'arb_opp_sell_y'],
      dtype='object')

reorder columns for better readibility

In [93]:
# reorder columns
def set_column_sequence(dataframe, seq, front=True):
    '''Takes a dataframe and a subsequence of its columns,
       returns dataframe with seq as first columns if "front" is True,
       and seq as last columns if "front" is False.
    '''
    cols = seq[:] # copy so we don't mutate seq
    for x in dataframe.columns:
        if x not in cols:
            if front: #we want "seq" to be in the front
                #so append current column to the end of the list
                cols.append(x)
            else:
                #we want "seq" to be last, so insert this
                #column in the front of the new column list
                #"cols" we are building:
                cols.insert(0, x)
    return dataframe[cols]

In [97]:
set_column_sequence(deduped_result_new_eliminate_zero, ['raw_pair_x', 'arb_opp_buy', 'arb_opp_sell', 'time_x',  'count', 
                         'max_high_price', 'min_low_price',
       'price_differential', '__time', 'marketplace', 'price_open', 'price_high',
       'price_low', 'price_close', 'price_vwap', 'trade_count', 'volume',
       'volume_avg', 'volume_max', 'volume_min', 'weekday', 'month', 'day',
       'year', 'time_y', '24_hour_time', 'time6', 'time7', 'time8',
       'raw_pair0', 'raw_pair1', 'arb_opp_y', 'arb_opp_buy_y',
       'arb_opp_sell_x',  'arb_opp_sell_y', 'arb_opp_x',  'arb_opp_buy_x', 'raw_pair_y'])

,raw_pair_x,arb_opp_buy,arb_opp_sell,time_x,count,max_high_price,min_low_price,price_differential,__time,marketplace,...,time8,raw_pair0,raw_pair1,arb_opp_y,arb_opp_buy_y,arb_opp_sell_x,arb_opp_sell_y,arb_opp_x,arb_opp_buy_x,raw_pair_y
77026,BTC/EUR,BTC/EUR Sun Jan 31 2021 17:00:00 GMT-0700 (Mou...,NaN,Sun Jan 31 2021 17:00:00 GMT-0700 (Mountain St...,6,9990000.0,26883.904,9.963116e+06,Sun Jan 31 2021 17:00:00 GMT-0700 (Mountain St...,coinbase,...,Time),BTC,EUR,BTC/EUR Sun Jan 31 2021 17:00:00 GMT-0700 (Mou...,NaN,BTC/EUR Sun Jan 31 2021 17:00:00 GMT-0700 (Mou...,BTC/EUR Sun Jan 31 2021 17:00:00 GMT-0700 (Mou...,BTC/EUR Sun Jan 31 2021 17:00:00 GMT-0700 (Mou...,NaN,BTC/EUR
76940,BTC/EUR,NaN,BTC/EUR Sun Jan 31 2021 17:00:00 GMT-0700 (Mou...,Sun Jan 31 2021 17:00:00 GMT-0700 (Mountain St...,6,9990000.0,26883.904,9.963116e+06,Sun Jan 31 2021 17:00:00 GMT-0700 (Mountain St...,itbit,...,Time),BTC,EUR,BTC/EUR Sun Jan 31 2021 17:00:00 GMT-0700 (Mou...,BTC/EUR Sun Jan 31 2021 17:00:00 GMT-0700 (Mou...,NaN,NaN,BTC/EUR Sun Jan 31 2021 17:00:00 GMT-0700 (Mou...,BTC/EUR Sun Jan 31 2021 17:00:00 GMT-0700 (Mou...,BTC/EUR
80740,BTC/EUR,BTC/EUR Sun Mar 07 2021 17:00:00 GMT-0700 (Mou...,NaN,Sun Mar 07 2021 17:00:00 GMT-0700 (Mountain St...,7,3693000.0,41702.758,3.651297e+06,Sun Mar 07 2021 17:00:00 GMT-0700 (Mountain St...,bittrex,...,Time),BTC,EUR,BTC/EUR Sun Mar 07 2021 17:00:00 GMT-0700 (Mou...,NaN,BTC/EUR Sun Mar 07 2021 17:00:00 GMT-0700 (Mou...,BTC/EUR Sun Mar 07 2021 17:00:00 GMT-0700 (Mou...,BTC/EUR Sun Mar 07 2021 17:00:00 GMT-0700 (Mou...,NaN,BTC/EUR
80655,BTC/EUR,NaN,BTC/EUR Sun Mar 07 2021 17:00:00 GMT-0700 (Mou...,Sun Mar 07 2021 17:00:00 GMT-0700 (Mountain St...,7,3693000.0,41702.758,3.651297e+06,Sun Mar 07 2021 17:00:00 GMT-0700 (Mountain St...,itbit,...,Time),BTC,EUR,BTC/EUR Sun Mar 07 2021 17:00:00 GMT-0700 (Mou...,BTC/EUR Sun Mar 07 2021 17:00:00 GMT-0700 (Mou...,NaN,NaN,BTC/EUR Sun Mar 07 2021 17:00:00 GMT-0700 (Mou...,BTC/EUR Sun Mar 07 2021 17:00:00 GMT-0700 (Mou...,BTC/EUR
54596,BTC/EUR,BTC/EUR Sat Jan 30 2021 17:00:00 GMT-0700 (Mou...,NaN,Sat Jan 30 2021 17:00:00 GMT-0700 (Mountain St...,7,1243940.6,26772.836,1.217168e+06,Sat Jan 30 2021 17:00:00 GMT-0700 (Mountain St...,bitfinex,...,Time),BTC,EUR,BTC/EUR Sat Jan 30 2021 17:00:00 GMT-0700 (Mou...,NaN,BTC/EUR Sat Jan 30 2021 17:00:00 GMT-0700 (Mou...,BTC/EUR Sat Jan 30 2021 17:00:00 GMT-0700 (Mou...,BTC/EUR Sat Jan 30 2021 17:00:00 GMT-0700 (Mou...,NaN,BTC/EUR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136777,BTT/BTC,NaN,BTT/BTC Wed Apr 29 2020 18:00:00 GMT-0600 (Mou...,Wed Apr 29 2020 18:00:00 GMT-0600 (Mountain Da...,2,3.3962266e-08,3e-08,3.962266e-09,Wed Apr 29 2020 18:00:00 GMT-0600 (Mountain Da...,bitfinex,...,Time),BTT,BTC,BTT/BTC Wed Apr 29 2020 18:00:00 GMT-0600 (Mou...,BTT/BTC Wed Apr 29 2020 18:00:00 GMT-0600 (Mou...,NaN,NaN,BTT/BTC Wed Apr 29 2020 18:00:00 GMT-0600 (Mou...,BTT/BTC Wed Apr 29 2020 18:00:00 GMT-0600 (Mou...,BTT/BTC
42174,BTT/BTC,NaN,BTT/BTC Mon Oct 19 2020 18:00:00 GMT-0600 (Mou...,Mon Oct 19 2020 18:00:00 GMT-0600 (Mountain Da...,2,3e-08,2.6666667e-08,3.333333e-09,Mon Oct 19 2020 18:00:00 GMT-0600 (Mountain Da...,bitfinex,...,Time),BTT,BTC,BTT/BTC Mon Oct 19 2020 18:00:00 GMT-0600 (Mou...,BTT/BTC Mon Oct 19 2020 18:00:00 GMT-0600 (Mou...,NaN,NaN,BTT/BTC Mon Oct 19 2020 18:00:00 GMT-0600 (Mou...,BTT/BTC Mon Oct 19 2020 18:00:00 GMT-0600 (Mou...,BTT/BTC
115206,BTT/BTC,NaN,BTT/BTC Tue Aug 18 2020 18:00:00 GMT-0600 (Mou...,Tue Aug 18 2020 18:00:00 GMT-0600 (Mountain Da...,2,4.1511413e-08,3.839356e-08,3.117853e-09,Tue Aug 18 2020 18:00:00 GMT-0600 (Mountain Da...,bittrex,...,Time),BTT,BTC,BTT/BTC Tue Aug 18 2020 18:00:00 GMT-0600 (Mou...,BTT/BTC Tue Aug 18 2020 18:00:00 GMT-0600 (Mou...,NaN,NaN,BTT/BTC Tue Aug 18 2020 18:00:00 GMT-0600 (Mou...,BTT/BTC Tue Aug 18 2020 18:00:00 GMT-0600 (Mou...,BTT/BTC
12108,BTT/BTC,NaN,BTT/BTC Fri Jun 19 2020 18:00:00 GMT-0600 (Mou...,Fri Jun 19 2020 18:00:00 GMT-0600 (Moun

In [100]:
deduped_result_new_eliminate_zero_show_columns = deduped_result_new_eliminate_zero[['time_x', 'raw_pair_x', 'price_differential', 'arb_opp_sell', 'arb_opp_buy', 'marketplace']]

In [101]:
deduped_result_new_eliminate_zero_show_columns

,time_x,raw_pair_x,price_differential,arb_opp_sell,arb_opp_buy
77026,Sun Jan 31 2021 17:00:00 GMT-0700 (Mountain St...,BTC/EUR,9.963116e+06,NaN,BTC/EUR Sun Jan 31 2021 17:00:00 GMT-0700 (Mou...
76940,Sun Jan 31 2021 17:00:00 GMT-0700 (Mountain St...,BTC/EUR,9.963116e+06,BTC/EUR Sun Jan 31 2021 17:00:00 GMT-0700 (Mou...,NaN
80740,Sun Mar 07 2021 17:00:00 GMT-0700 (Mountain St...,BTC/EUR,3.651297e+06,NaN,BTC/EUR Sun Mar 07 2021 17:00:00 GMT-0700 (Mou...
80655,Sun Mar 07 2021 17:00:00 GMT-0700 (Mountain St...,BTC/EUR,3.651297e+06,BTC/EUR Sun Mar 07 2021 17:00:00 GMT-0700 (Mou...,NaN
54596,Sat Jan 30 2021 17:00:00 GMT-0700 (Mountain St...,BTC/EUR,1.217168e+06,NaN,BTC/EUR Sat Jan 30 2021 17:00:00 GMT-0700 (Mou...
...,...,...,...,...,...
136777,Wed Apr 29 2020 18:00:00 GMT-0600 (Mountain Da...,BTT/BTC,3.962266e-09,BTT/BTC Wed Apr 29 2020 18:00:00 GMT-0600 (Mou...,NaN
42174,Mon Oct 19 2020 18:00:00 GMT-0600 (Mountain Da...,BTT/BTC,3.333333e-09,BTT/BTC Mon Oct 19 2020 18:00:00 GMT-0600 (Mou...,NaN
115206,Tue Aug 18 2020 18:00:00 GMT-0600 (Mountain Da...,BTT/BTC,3.117853e-09,BTT/BTC Tue Aug 18 2020 18:00:00 GMT-0600 (Mou...,NaN
12108,Fri Jun 19 2020 18:00:00 GMT-0600 (Mountain Da...,BTT/BTC,2.894776e-09,BTT/BTC Fri Jun 19 2020 18:00:00 GMT-0600 (Mou...,NaN


In [ ]:
arb_opp_sell and arb_opp_buy marks opportunities to sell and buy for arbitrage opportunities
i.e. looking at the first two rows, the action to take is to sell BTC/EUR at itbase and buy at coinbase

#### Appendix - Ignore

In [42]:
### get max and min of the group bys 
price_highs_grouped_by_ticker_time = df_pairs.groupby(['__time','raw_pair0', 'raw_pair1']).agg({'price_high': ['max', 'min']})

In [45]:
price_highs_grouped_by_ticker_time.to_csv('price_highs_grouped_by_ticker_time.csv')

In [41]:
price_lows_grouped_by_ticker_time = df_pairs.groupby(['__time','raw_pair0', 'raw_pair1']).agg({'price_low': ['max', 'min']})

In [44]:
price_lows_grouped_by_ticker_time.to_csv('price_lows_grouped_by_ticker_time.csv')

In [12]:
df_pairs.groupby(['raw_pair0', 'raw_pair1', 'marketplace']).mean().to_csv('grouped_by_ticker_marketplace.csv')

In [13]:
df_pairs.groupby(['raw_pair0', 'raw_pair1', 'marketplace', 'year']).mean().to_csv('grouped_by_ticker_marketplace_year.csv')

In [14]:
grouped_tickers = df_pairs.groupby(['raw_pair0', 'raw_pair1'], as_index = False).mean()

In [16]:
grouped_tickers.raw_pair0.value_counts()

USDT        29
BTC         28
ETH         21
BNB         19
XRP         18
            ..
POT          1
QNT          1
GUP          1
LINKDOWN     1
XTP          1
Name: raw_pair0, Length: 568, dtype: int64

In [101]:
tickers = ['USDT', 'BTC', 'ETH', 'BNB', 'XRP', 'TRX', 'LINK', 'EOS', 'NEO', 'LTC', 'BCH', 'ZEC', 'XLM', 'ALGO', 'USD']
grouped_tickers[grouped_tickers['raw_pair0'].str.contains('|'.join(tickers)) | grouped_tickers['raw_pair1'].str.contains('|'.join(tickers))].to_csv('above_10_occurances.csv')

#### Variables to arbitrage
#1. Day of the week
#2. Month
#3. Day of the month
#4. Year
#5. Daylight saving zones
#6. pairs
#7. marketplace


In [21]:
df['marketplace']=df['marketplace'].astype('category').cat.codes
df['raw_pair0']=df['raw_pair0'].astype('category').cat.codes
df['raw_pair1']=df['raw_pair1'].astype('category').cat.codes

In [18]:
tickers = ['BTC/ETH', 'ETH/BTC']
df_pairs[df_pairs['raw_pair'].str.contains('|'.join(tickers))]

,__time,marketplace,raw_pair,price_open,price_high,price_low,price_close,price_vwap,trade_count,volume,...,month,day,year,time,24_hour_time,time6,time7,time8,raw_pair0,raw_pair1
290,Fri Mar 15 2019 18:00:00 GMT-0600 (Mountain Da...,binance,ETH/BTC,0.034882,0.035487,0.034882,0.035241,0.035231,178627,258855.6700,...,Mar,15,2019,18:00:00,GMT-0600,(Mountain,Daylight,Time),ETH,BTC
295,Fri Mar 15 2019 18:00:00 GMT-0600 (Mountain Da...,bitfinex,ETH/BTC,0.034883,0.035449,0.034883,0.035260,0.035287,4760,13520.2160,...,Mar,15,2019,18:00:00,GMT-0600,(Mountain,Daylight,Time),ETH,BTC
301,Fri Mar 15 2019 18:00:00 GMT-0600 (Mountain Da...,bitstamp,ETH/BTC,0.034899,0.035459,0.034899,0.035265,0.035266,749,3694.1702,...,Mar,15,2019,18:00:00,GMT-0600,(Mountain,Daylight,Time),ETH,BTC
304,Fri Mar 15 2019 18:00:00 GMT-0600 (Mountain Da...,coinbase,ETH/BTC,0.034879,0.035632,0.034879,0.035260,0.035348,6876,13549.5490,...,Mar,15,2019,18:00:00,GMT-0600,(Mountain,Daylight,Time),ETH,BTC
309,Fri Mar 15 2019 18:00:00 GMT-0600 (Mountain Da...,gemini,ETH/BTC,0.035520,0.035570,0.034660,0.034660,0.035487,290,2350.9844,...,Mar,15,2019,18:00:00,GMT-0600,(Mountain,Daylight,Time),ETH,BTC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1166286,Sat Mar 13 2021 17:00:00 GMT-0700 (Mountain St...,coinbase,ETH/BTC,0.031334,0.031334,0.030991,0.031229,0.031127,23428,17303.4510,...,Mar,13,2021,17:00:00,GMT-0700,(Mountain,Standard,Time),ETH,BTC
1166725,Sat Mar 13 2021 17:00:00 GMT-0700 (Mountain St...,gemini,ETH/BTC,0.031347,0.031347,0.030983,0.031188,0.031129,513,1544.7507,...,Mar,13,2021,17:00:00,GMT-0700,(Mountain,Standard,Time),ETH,BTC
1166728,Sat Mar 13 2021 17:00:00 GMT-0700 (Mountain St...,kraken,ETH/BTC,0.031349,0.031349,0.030949,0.031169,0.031138,4662,7074.7330,...,Mar,13,2021,17:00:00,GMT-0700,(Mountain,Standard,Time),ETH,BTC
1166739,Sat Mar 13 2021 17:00:00 GMT-0700 (Mountain St...,poloniex,ETH/BTC,0.031344,0.031344,0.031040,0.031194,0.031156,4818,11730.2690,...,Mar,13,2021,17:00:00,GMT-0700,(Mountain,Standard,Time),ETH,BTC
